# How to Add a New Game

### Disclaimer
This notebook closely follows the tutorial steps from [docs/howto_add_games.md](https://github.com/clp-research/clembench/blob/main/docs/howto_add_games.md)
 in the [**clp-research/clembench**](https://github.com/clp-research/clembench) repository. <br> 
While the core content is derived from the original tutorial, some additional detailed descriptions and explanations have been added to serve as a source of knowledge during the development of our project.

## The GameBenchMark Class
The Benchmark is run for a particular game with **python scripts/cli.py run -g game -m model** <br>
- If only one model is defined for a two player game, it will represent both players.

## Execution
- When the run command is executed, the run routine in `benchmark.py` will determine the game code that needs to be invoked.
- The benchmark code loads all subclasses that inherit from GameBenchMark and calls `setup` on them.
- The setup method already loads the game instances **-->** every subclass is asked if it applies to the given name.
- That means, there has to be a subclass like the following for each game
- The subclass provides `GAME_NAME=taboo` and the rest is taken care of by the SuperClass
- Then the benchmark code checks if your game is single or multiplayer game (the default is multi-player).
- Then the run(dialog_pair,temperature) method is called by GameBenchmark.
- This is when the GameMaster becomes relevant (which is returned by thr `create_game_master()` factory method).

In [ ]:
class TabooGameBenchmark(GameBenchmark):

    def __init__(self):
        super().__init__(GAME_NAME)

    def get_description(self):
        return "Taboo game between two agents where one has to describe a word for the other to guess."

    def create_game_master(self, experiment: Dict, player_backends: List[str]) -> GameMaster:
        return Taboo(experiment, player_backends)
        
    def is_single_player(self) -> bool:
        return False

#### `get_description method`
This is returned when `python3 scripts/cli.py ls` is executed, see: <br>
```
2024-08-31 12:55:58,735 - benchmark.run - INFO - Listing benchmark games:
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: cloudgame -> A simple game in which a player has to decide whether they see clouds or not.
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: imagegame -> Image Game simulation to generate referring expressions and fill a grid accordingly
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: matchit -> A simple game in which two players have to decide whether they see the same image or not.
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: matchit -> A simple game in which two players have to decide whether they see the same image or not.
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: matchit -> A simple game in which two players have to decide whether they see the same image or not.
2024-08-31 12:55:58,736 - benchmark.run - INFO -  Game: matchit -> A simple game in which two players have to decide whether they see the same image or not.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: privateshared -> Questioner and answerer in scorekeeping game.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: taboo -> Taboo game between two agents where one has to describe a word for the other to guess.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: textmapworld -> Graph Game.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: textmapworld_description -> Graph Game.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: textmapworld_graphreasoning -> Graph Game.
2024-08-31 12:55:58,737 - benchmark.run - INFO -  Game: textmapworld_questions -> Graph Game.
2024-08-31 12:55:58,738 - benchmark.run - INFO -  Game: textmapworld_specificroom -> Graph Game.
```

## GameMaster Class
- For each `experiment`in instances.json, that has been loaded with `on_setup()`, the game benchmark code applies the given dialog pair (or if not provided, tries to determine it based on instance information) <br>
- Each experiment represents a specific condition for the game, like difficulity and holds the actual game instance. <br>
- A `GameMaster` is created for each instance, by `self.create_gamemaster` method of the BenchMark. <br>
- The `GameMaster`is in charge of playing a single isntance of the game <br>
- For Taboo: Target word to be guessed, words that are not allowed to be said.

In [ ]:
try:
   game_master = self.create_game_master(experiment_config, dialogue_pair)
   # Recieve Game Information
   game_master.setup(**game_instance)
   # Coordinate actual Play
   game_master.play()
   # Stores the interactions between the players and GM in game_record_dir
   game_master.store_records(game_id, game_record_dir)
except Exception:  # continue with other episodes if something goes wrong
   self.logger.exception(f"{self.name}: Exception for episode {game_id} (but continue)")

## Relevant Classes

#### MyBenchMark - extending GameBenchMark
Following Methods are necessary to implement: <br>
- `__init__(self)`, that calls on `super().__init__(GAME_NAME)`
- `get_description`- explained above
- `is_single_player(self) -> bool` - determines if one player is sufficient
- `create_game_master(self, experiment: Dict, player_backends: List[str]) -> MyGameMaster` that returns **MyGameMaster**
<br>

### MyGameMaster that extends GameMaster
- `__init__(self, name: str, experiment: Dict, player_backends: List[str] = None):` 
    - Receives the experiment information and the players that play the game.
    - These can be simply delegated to `super()`.
- `setup(self, **game_instance)` which sets the information specified in `instances.json`
- `play(self)`which executes the game logic and performs turns in the game
- `compute_scores` which is called when the user executes `python3 scripts/cli.py score taboo`

## DialogueGameMaster
`MyGameMaster` can implement play(), but in some cases we can extend from `DialogueGameMaster`, which is a more concrete subclass of `GameMaster`. <br>
It defines the play routine as: 

In [ ]:
def play(self) -> None:
     self._on_before_game()
     while self._does_game_proceed():
         self.log_next_turn()  # not sure if we want to do this always here (or add to _on_before_turn)
         self._on_before_turn(self.current_turn)
         self.logger.info(f"{self.name}: %s turn: %d", self.name, self.current_turn)
         for player in self.__player_sequence():
             if not self._does_game_proceed():
                 break  # potentially stop in between player turns
             # GM -> Player
             history = self.messages_by_names[player.descriptor]
             assert history, f"messages history must not be empty for {player.descriptor}"

             last_entry = history[-1]
             assert last_entry["role"] != "assistant", "Last entry should not be assistant " \
                                                       "b.c. this would be the role of the current player"
             message = last_entry["content"]

             action = {'type': 'send message', 'content': message}
             self.log_event(from_='GM', to=player.descriptor, action=action)

             _prompt, _response, response_message = player(history, self.current_turn)

             # Player -> GM
             action = {'type': 'get message', 'content': response_message}
             self.log_event(from_=player.descriptor, to="GM", action=action, call=(_prompt, _response))

             # GM -> GM
             self.__validate_parse_and_add_player_response(player, response_message)
         self._on_after_turn(self.current_turn)
         self.current_turn += 1
     self._on_after_game()

As long as `_does_game_proceed():`
- **GM --> Player:**
    1. At a player's turn, the player recieves its view on the history of messages (`messages_by_names`)
    2. The last message is logged (`log_event`) as a GM->Player event in the interactions log. 
    3. Then player is asked to create a response based on the history and current turn index.
- **Player --> Game:**
    1. The response is recieved and logged as Player --> GM in the event log
- **GM --> GM**
    1. Validates and stores response if valid, then goes to next turn

This shows that the logging is systematically done when using DialogueGameMaster. <br>
There are however several ways to customize the gameplay:

- `def _on_setup(self, **kwargs)` which must be implemented. Use add_player() here to add the players.
- `def _does_game_proceed(self) -> bool` which must be implemented. Decides if the game can continue.
- `def _validate_player_response(self, player: Player, utterance: str) -> bool` to decide if an utterance should be added. This is also the place to check for game end conditions.
- `def _on_parse_response(self, player: Player, utterance: str) -> Tuple[str, bool]` to decide if a response utterance should be modified. If not simply return the utterance.
- `def _after_add_player_response(self, player: Player, utterance: str)` to add the utterance to other player's history, if necessary. To do this use the method add_user_message(other_player,utterance).
- the general game hooks `_on_before_game()` and `_on_after_game()`
- the general turn hooks `_on_before_turn(turn_idx)` and `_on_after_turn(turn_idx)`

### Setup Taboo Game
The setup hook is used to set instance speciifc values and to setup the **WordDescriber** and **WordGuesser** which are the Player for the game.

In [ ]:
def _on_setup(self, **game_instance):
    logger.info("_on_setup")
    self.game_instance = game_instance

    self.describer = WordDescriber(self.player_models[0], self.max_turns)
    self.guesser = WordGuesser(self.player_models[1])

    self.add_player(self.describer)
    self.add_player(self.guesser)

# General game hook is used to set the initial prompts for both players
def _on_before_game(self):
  self.add_user_message(self.describer, self.describer_initial_prompt)
  self.add_user_message(self.guesser, self.guesser_initial_prompt)

# Then it has to be decided if the guessing should continue
def _does_game_proceed(self):
    if self.invalid_response:
        self.log_to_self("invalid format", "abort game")
        return False
    if self.clue_error is not None:
        return False 
    if self.current_turn >= self.max_turns:
        self.log_to_self("max turns reached", str(self.max_turns))
        return False
    return True

# Then check if the player response is in valid format. (MOVE_RULE)
def _validate_player_response(self, player, utterance: str) -> bool:
  if player == self.guesser:
      if not utterance.startswith("GUESS:"):
          self.invalid_response = True
          return False
  if player == self.describer:
      if not utterance.startswith("CLUE:"):
          self.invalid_response = True
          return False
      errors = check_clue(utterance, self.target_word, self.related_words)
      if errors:
          error = errors[0]
          self.clue_error = error
          return False
  self.log_to_self("valid format", "continue")
  return True

# This is where we can detect invalid MOVEs or log reponses without prefixes:
def _on_parse_response(self, player, utterance: str) -> Tuple[str, bool]:
  if player == self.guesser:
      utterance = utterance.replace("GUESS:", "")
      self.guess_word = utterance.lower()
      self.log_to_self("guess", self.guess_word)
  if player == self.describer:
      utterance = utterance.replace("CLUE:", "")
      self.log_to_self("clue", utterance)
  return utterance, False

# The (modified) response is then added to the player's history:
def _after_add_player_response(self, player, utterance: str):
    if player == self.describer:
        utterance = f"CLUE: {utterance}."
        self.add_user_message(self.guesser, utterance)
    if player == self.guesser:
        if self.guess_word != self.target_word:
            utterance = f"GUESS: {self.guess_word}."
            self.add_user_message(self.describer, utterance)

# Finally, use general turn method to additionally log the initial prompt for the second player 
# and not only the most recent one (as automatically done by GameMaster)
def _on_before_turn(self, turn_idx: int):
    if turn_idx == 0:
        self.log_message_to(self.guesser, self.guesser_initial_prompt)

## GameResourceLocator
Provides methods to access, load and store files from within the game directory. <br>
When implementing the game, always use the methods of this class to handle files.
- **Usage:** 
    - `gm.load_json("my_file")` located directly in the game's directory
    - `gm.load_json("sub/my_file")` in `game/sub/my_file.json` to acces subdirectories.

**Expected Game Structure:**
~~~
games
├──mygame
│     ├── in
│     │   └── instances.json
│     ├── resources
│     │   └── initial_prompt.template
│     ├── instancegenerator.py
│     └── master.py
...
~~~

## Player Class
A player object recieves messages and returns a textual response. <br>
- The response can be either from an api to a cLLM with the `__call__`method, or a predefined `_custom_response`

In [ ]:
from clemgame.clemgame import Player

class WordGuesser(Player):

   def __init__(self, model_name):
      super().__init__(model_name)

   def _custom_response(self, messages, turn_idx):
      # mock response
      return f'Pear'

## GameInstanceGenerator
- To let agents play a game, we need to have a description that instantiates single episodes
- **Taboo:** Each episode is played with a specific target word that also comes with a list of other, related and forbidden words.
- The Class generates full instances that include initial propmts for the models and other metha information for running experiments.
- **Example for Taboo:**
    - Use word list of 3 frequencies (low/medium/high)
    - Test 3 LLMs (Taboo is played by 2 LLMs)
    - Fix the number of maximum turns
    - Generate fix number of instances
- Instances are generated as a JSON in `games/game_name/in/instances.json`

In [ ]:
from clemgame.clemgame import GameInstanceGenerator

N_INSTANCES = 20  # how many different target words; zero means "all"
N_GUESSES = 3  # how many tries the guesser will have
N_REATED_WORDS = 3
LANGUAGE = "en"

class TabooGameInstanceGenerator(GameInstanceGenerator):

    def __init__(self):
        super().__init__("taboo")

    def on_generate(self):
        player_assignments = list(itertools.permutations([OpenAI.MODEL_GPT_35, Anthropic.MODEL_CLAUDE_13]))
        for difficulty in ["low", "medium", "high"]:

            # first choose target words based on the difficultly
            fp = f"resources/target_words/{LANGUAGE}/{difficulty}_freq_100"
            target_words = self.load_file(file_name=fp, file_ending=".txt").split('\n')
            if N_INSTANCES > 0:
                assert len(target_words) >= N_INSTANCES, \
                    f'Fewer words available ({len(target_words)}) than requested ({N_INSTANCES}).'
                target_words = random.sample(target_words, k=N_INSTANCES)

            # use the same target_words for the different player assignments
            experiment = self.add_experiment(f"{difficulty}_{LANGUAGE}", dialogue_partners=player_assignments)
            experiment["max_turns"] = N_GUESSES

            describer_prompt = self.load_template("resources/initial_prompts/initial_describer")
            guesser_prompt = self.load_template("resources/initial_prompts/initial_guesser")
            experiment["describer_initial_prompt"] = describer_prompt
            experiment["guesser_initial_prompt"] = guesser_prompt

            for game_id in tqdm(range(len(target_words))):
                target = target_words[game_id]

                game_instance = self.add_game_instance(experiment, game_id)
                game_instance["target_word"] = target
                game_instance["related_word"] = []

                if len(game_instance["related_word"]) < N_REATED_WORDS:
                    print(f"Found less than {N_REATED_WORDS} related words for: {target}")

## Running Experiments with games
- Adding the `-e` argument to the execution of `cli.py` declares that an experiment is to be done.
- This creates a **results** folder with the following structure: <br>
    - Directories that mention the involved models
        - i.e.: gpt-3.5-turbo-1106-t0.0--gpt-3.5-turbo-1106-t0.0
    - Directory structure for each episode (based on instances.json):
        - **instance.json**
        - **interaction.json**
        - **transcript.html**
    - experiment_name.json, that contains the run parameters

## Huggingface Prototyping Check Methods
The huggingface-local backend offers two functions to check messages lists that clemgames might pass to the backend without the need to load the full model weights.  <br>
Allows to prototype clemgames locally with minimal hardware demand.

### Messages Checking
The `check_messages` function in `backends/huggingface_local_api.py` takes a messages list and a ModelSpec as arguments. <br>
- Prints all anticipated issues with the passed messages list to console if they occur. <br> 
- Applies the given model's chat template to the messages as a direct check. <br>
- Returns **False** if the chat template does not accept the messages and prints the outcome to console.

### Context Limit Checking
The `check_context_limit` takes a messages list and a ModelSpec as required arguments.
- Further arguments:
    - number of tokens to generate max_new_tokens: int (default: 100), 
    - clean_messages: bool (default: False) to apply message cleaning as the generation method will,
    - verbose: bool (default: True) for console printing of the values.
- Prints:
    - The token count for the passed messages after chat template application,
    - The remaining number of tokens (negative if context limit is exceeded)
    - Maximum number of tokens the model allows as generation input.
- Returns a tuple with four elements:
    - **bool:** True if context limit was not exceeded, False if it was.
    - **int:** number of tokens for the passed messages.
    - **int:** number of tokens left in context limit.
    - **int:** context token limit.